In [1]:
# Use a pipeline as a high-level helper
from transformers import pipeline
import os

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"

# pipe1 = pipeline("text-generation", model="/home/ai2lab/weihua/2024_IR_Law_Project/LLaMA-Factory/models/tw_llama_step2", device_map="auto")

pipe = pipeline("text-generation", model="./LLaMA-Factory/models/tw_llama_step1", device_map="auto")


/home/ai2lab/anaconda3/envs/qwen2/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 9/9 [00:13<00:00,  1.52s/it]


In [2]:
import json

all_test_data = []

with open('./data/test_data.jsonl', 'r') as file:
    for line in file:
        # data_str = '{"id":"test_0","title":"外籍勞工在合約到期後如何變更雇主?","question":"各位好,請教一下,如果外籍勞工在合約期滿後想要更換雇主,現雇主是否有權不允許其轉換雇主,只是提供兩個選擇:一是繼續留任,二是必須回國,這樣的做法是否違反法律?"}'
        data_str = line

        start_id = data_str.find('"id":"') + len('"id":"')
        end_id = data_str.find('","title"')
        id_value = data_str[start_id:end_id]

        # 提取 title
        start_title = data_str.find('"title":"') + len('"title":"')
        end_title = data_str.find('","question"')
        title = data_str[start_title:end_title]

        # 提取 question
        start_question = data_str.find('"question":"') + len('"question":"')
        end_question = data_str.rfind('"')
        question = data_str[start_question:end_question]

        all_test_data.append((id_value, title, question))

In [3]:
import re

output_data = {'id':[], 'TARGET':[]}

for idx, line in enumerate(all_test_data):
        
    output_data['id'].append(line[0])
    
    query = line[1]+','+line[2]
    messages = [
        # {"role": "user", "content": "回答法條時，只能列出以下法條名稱[駐外機構組織通則,公寓大廈管理條例,農業發展條例,法務部組織法,家事事件法審理細則,補習及進修教育法,消費者保護法,民事訴訟法,高級中等教育法,海岸巡防法,電子簽章法,嚴重特殊傳染性肺炎防治及紓困振興特別條例,信用卡業務機構管理辦法,外國護照簽證條例,爆竹煙火製造儲存販賣場所設置及安全管理辦法,教師待遇條例,職業安全衛生法,直轄市縣（市）不動產糾紛調處委員會設置及調處辦法,勞工退休準備金提撥及管理辦法,基本工資審議辦法,勞動事件法,偵查不公開作業辦法,國民年金法,羈押法,農田水利法,動物保護法,監獄行刑法,強制執行法,通訊保障及監察法,廢棄物清理法,軍事審判法,各級法院辦理民事事件遠距訊問作業辦法,私立學校法,法律扶助法,姓名條例,中華民國憲法增修條文,全民健康保險法施行細則,教師法施行細則,司法院組織法,鄉鎮市調解條例,各級法院法官自律實施辦法,中高齡者及高齡者就業促進法,藥事法,人民團體法,行刑累進處遇條例,職工福利委員會組織準則,傳染病防治法,地政士法,優生保健法施行細則,毒品危害防治條例,戶籍法,不動產經紀業管理條例,訴願法,房屋稅條例,中華民國刑法,公務人員任用法,營養師法,著作財產權人不明著作利用許可申請規費收費準則,行政訴訟法,食品安全衛生管理法,師資培育法,中華民國國徽國旗法,高級中等學校學生學籍管理辦法,原住民族傳統智慧創作保護條例,家庭暴力防治法 ,菸酒管理法,性別平等工作法,個人資料保護法,勞動基準法檢舉案件保密及處理辦法,商品標示法,公民投票法,洗錢防制法,道路交通安全規則,電子遊戲場業管理條例,票據法,職業安全衛生管理辦法,刑事訴訟遠距訊問作業辦法,政治獻金法,土地法,商標法,法務部調查局組織法,會計師查核簽證公司登記資本額辦法,戶籍法施行細則,違反道路交通管理事件統一裁罰標準及處理細則,一般爆竹煙火型式認可及個別認可作業辦法,犯罪被害人權益保障法,發展觀光條例,電信法,駐外使領館組織條例,中華民國刑法施行法 ,校園霸凌防制準則,著作權集體管理團體條例,就業保險促進就業實施辦法,非訟事件法,組織犯罪防制條例,性騷擾事件調解辦法,社會秩序維護法,勞工保險條例,性騷擾防治法施行細則,專科以上學校產學合作實施辦法,師資培育之大學及教育實習機構辦理教育實習辦法,行政程序法,會計師查核簽證有限合夥登記出資額辦法,文化資產保存法,強制汽車責任保險法,遺產及贈與稅法,土地徵收條例,區域計畫法,警察職權行使法,行政執行法,證券交易法,刑事補償法,觀光旅館業管理規則,保險法,家事事件審理細則,道路交通事故處理辦法,公務人員考試法,立法院職權行使法,高級中等以下學校兼任代課及代理教師聘任辦法,醫師法施行細則,貪污治罪條例,司法院大法官自律實施辦法,短期補習班設立及管理準則,民法總則施行法,勞動基準法,特定寵物業管理辦法,人口販運防制法,精神衛生法,學生輔導法,陸海空軍刑法,入出國及移民法,兒童及少年福利與權益保障法,計程車駕駛人執業登記管理辦法,監獄行刑法施行細則,醫事人員執業登記及繼續教育辦法,跟蹤騷擾防制法,高速公路及快速公路交通管制規則,通訊交易解除權合理例外情事適用準則,大量解僱勞工保護法,違反社會秩序維護法案件處理辦法,民宿管理辦法,性騷擾防治法,稅捐稽徵法, 公職人員選舉罷免法,中央政府機關總員額法,工會法,森林法,公職人員選舉罷免法,勞工保險條例施行細則,菸害防制法,憲法訴訟法,汽車運輸業管理規則,租賃住宅市場發展及管理條例,防空演習實施辦法,災害防救法,國家公園法,公寓大廈管理條例施行細則,涉外民事法律適用法,地方制度法,所得稅法,全民健康保險法,職業安全衛生教育訓練規則,法官法,公路法,臺灣地區與大陸地區人民關係條例,農產品市場交易法,國民法官法,公司法,公益勸募條例,銀行法,私立高級中等以上學校退場條例,具有多重醫事人員資格者執業管理辦法,安寧緩和醫療條例,刑事妥速審判法,市區道路條例,土地登記規則,當舖業法,道路交通管理處罰條例,郵件處理規則,教師請假規則,法院辦理民事事件證人鑑定人日費旅費及鑑定費支給標準,警械使用條例,動物傳染病防治條例,停車場法,法院組織法,土地稅法,就業保險法,提存法,營業秘密法,兒童及少年性剝削防制條例,性別工作平等法施行細則,律師法施行細則,民事閱卷規則,都市計畫法,民用航空法,車輛行車事故鑑定及覆議作業辦法,郵政法,爆竹煙火管理條例,文化創意產業發展法,性騷擾防治準則,公司名稱及業務預查審核準則,雇主聘僱外國人許可及管理辦法,公務人員週休二日實施辦法,集會遊行法,著作財產權人不明著作利用之許可授權及使用報酬辦法,勞工請假規則, 性別平等教育法,行政法院組織法,勞工退休金條例,人體器官移植條例,統一發票使用辦法,少年事件處理法,刑事訴訟法,都市更新會設立管理及解散辦法,醫師法,行政罰法,人類免疫缺乏病毒傳染防治及感染者權益保障條例,中央選舉委員會組織法,勞工健康保護規則,破產法,消防法,爆竹煙火管理條例施行細則,勞工保險條例 ,兵役法施行法,公證法,槍砲彈藥刀械管制條例,執行死刑規則,勞動檢查法,所得稅法施行細則,個人資料保護法施行細則,無障礙住宅設計基準及獎勵辦法,教師法,職工福利金條例,總統副總統選舉罷免法,空氣污染防制法,仲裁法,家庭暴力防治法,統計法,典試法,勞動檢查法施行細則,大陸地區人民在臺灣地區取得設定或移轉不動產物權許可辦法,法院辦理民事事件訴訟文書之影印攝影抄錄及翻譯費徵收標準,道路交通標誌標線號誌設置規則,農業產銷班設立及輔導辦法,醫療器材管理法,公平交易法,教育基本法,提審法,中央銀行法,強迫入學條例,存款帳戶及其疑似不法或顯屬異常交易管理辦法,消費者保護法施行細則,警察刑事紀錄證明核發條例,旅館業管理規則,家庭暴力防治法施行細則,國家賠償法,化粧品衛生安全管理法,全民健康保險醫療辦法,家事事件法,醫療法,野生動物保育法,有限合夥法,財團法人法,鐵路法,噪音管制法,證人保護法,勞工職業災害保險及保護法,農田水利會組織通則,建築法,勞資爭議處理法,兵役法,民防法,民事事件委任非律師為訴訟代理人許可準則,就業服務法施行細則,圖書館法,人工生殖法,勞動基準法施行細則,優生保健法,消除對婦女一切形式歧視公約施行法,中央法規標準法,經濟部對受嚴重特殊傳染性肺炎影響發生營運困難產業事業紓困振興辦法,懲治走私條例,遺產及贈與稅法施行細則,國民教育法,加值型及非加值型營業稅法,民法債編施行法,民法,職業災害勞工保護法,病人自主權利法,就業服務法,職業安全衛生法施行細則,商標法施行細則,公務人員請假規則,毒品危害防制條例,學位授予法,中華民國憲法,反滲透法,紀念日及節日實施辦法,性侵害犯罪防治法]"},
        {"role": "user", "content": f"判斷以下情境，與台灣那些法律條文相關: {query}"},
    ]
    result1 = pipe(messages)
    query2 = result1[0]['generated_text'][1]['content']
    
    # print(query2)
    output_data['TARGET'].append(query2)
    print(idx, query2)

Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


0 就業服務法第43條,就業服務法第59條,就業服務法第63條,就業服務法第73條,就業服務法第73條之1,就業服務法第73條之2
1 民法第1055條
2 勞工保險條例第11條,全民健康保險法第15條,全民健康保險法第16條,勞工保險條例第11條之1,勞工保險條例第11條之2,勞工保險條例第11條之3,勞工保險條例第11條之4,勞工保險條例第11條之5,勞工保險條例第11條之6,勞工保險條例第11條之7,勞工保險條例第11條之8,勞工保險條例第11條之9,勞工保險條例第11條之10,勞工保險條例第11條之11,勞工保險條例第11條之12,勞工保險條例第11條之13,勞工保險條例第11條之14,勞工保險條例第11條之15,勞工保險條例第11條之16,勞工保險條例第11條之17,勞工保險條例第11條之18,勞工保險條例第11條之19,勞工保險條例第11條之20,勞工保險條例第11條之21,勞工保險條例第11條之22,勞工保險條例第11條之23,勞工保險條例第11條之24,勞工保險條例第11條之25,勞工保險條例第11條之26,勞工保險條例第11條之27,勞工保險條例第11條之28,勞工保險條例第11條之29,勞工保險條例第11條之30,勞工保險條例第11條之31,勞工保險條例第11條之32,勞工保險條例第11條之33,勞工保險條例第11條之34,勞工保險條例第11條之35,勞工保險條例第11條之36,勞工保險條例第11條之37,勞工保險條例第11條之38,勞工保險條例第11條之39,勞工保險條例第11條之40,勞工保險條例第11條之41,勞工保險條例第11條之42,勞工保險條例第11條之43,勞工保險條例第11條之44,勞工保險條例第11條之45,勞工保險條例第11條之46,勞工保險條例第11條之47,勞工保險條例第11條之48,勞工保險條例第11條之49,勞工保險條例第11條之50,勞工保險條例第11條之51,勞工保險條例第11條之52,勞工保險條例第11條之53,勞工保險條例第11條之54,勞工保險條例第11條之55,勞工保險條例第11條之56,勞工保險條例第11條之57,勞工保險條例第11條之58,勞工保險條例第11條之59,勞工保險條例第11條之60,勞工保險條例第11條之61,勞工保險條例第11條之62,勞工保險條例第11條之63,勞工保險條例第11條之64

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


9 民法第12條,民法第14條,民法第15條,民法第16條
10 中華民國刑法第309條
11 民法第423條,民法第424條
12 勞動基準法第22條
13 勞動基準法第32條,勞動基準法第24條
14 民法第1055條
15 個人資料保護法第19條,個人資料保護法第20條,個人資料保護法第21條
16 公司法第15條,公司法第23條
17 中華民國憲法第14條,集會遊行法第6條,集會遊行法第7條
18 刑事訴訟法第33條,刑事訴訟法第31條
19 民法第184條,民法第188條,民法第193條,民法第195條,醫療法第63條,醫療法第81條
20 民法第24條,民法第41條,民法第42條,民法第43條,民法第44條,民法第45條,民法第47條,民法第48條,民法第49條,民法第50條,民法第51條,民法第52條,民法第53條,民法第54條,民法第55條,民法第56條,民法第57條,民法第58條,民法第59條,民法第60條,民法第61條,民法第62條,民法第63條,民法第64條,民法第65條,民法第66條,民法第67條,民法第68條,民法第69條,民法第70條,民法第71條,民法第72條,民法第73條,民法第74條,民法第75條,民法第76條,民法第77條,民法第78條,民法第79條,民法第80條,民法第81條,民法第82條,民法第83條,民法第84條,民法第85條,民法第86條,民法第87條,民法第88條,民法第89條,民法第90條,民法第91條,民法第92條,民法第93條,民法第94條,民法第95條,民法第96條,民法第97條,民法第98條,民法第99條,民法第100條,民法第101條,民法第102條,民法第103條,民法第104條,民法第105條,民法第106條,民法第107條,民法第108條,民法第109條,民法第110條,民法第111條,民法第112條,民法第113條,民法第114條,民法第115條,民法第116條,民法第117條,民法第118條,民法第119條,民法第120條,民法第121條,民法第122條,民法第123條,民法第124條,民法第125條,民法第126條,民法第127條,民法第128條,民法第129條,民法第130條,民法第131條,民法第132條,民法第133條,民法第134條,民法第135條,民法第136條,民法第137條,民法第1

In [4]:
import pandas as pd
df = pd.DataFrame(output_data)
df.to_csv('./result.csv', index=False)  